In [ ]:
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.utils import shuffle

lines = []
i = 0
with open('driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        if(i > 0): #this part is added to insure the 1st line in the csvfile which include the headers is not included
             lines.append(line)
        i = i + 1

# create adjusted steering measurements for the side camera images
correction = 0.01 # this is a parameter to tune

images = []
measurments = []
# use this section if you wanna use images from the 3-cameras
#for line in lines:
    #print(line)
 #   for i in range(3):
        # get the image path from the file
  #      source_path = line[i]
   #     filename = source_path.split('/')[-1]
    #    current_path = 'IMG/'+ filename
        #print('current_path', current_path)
     #   image = cv2.imread(current_path)
      #  images.append(image)
        # get the steering measurmement (labels) from the file
       # measurment = line[3]
        #if (i == 0):
         #   measurments.append(float(measurment))
        #if (i == 1):
         #    measurments.append(float(measurment) + correction)
        #if (i == 2):
         #    measurments.append(float(measurment) - correction)
            
            
  

# use this section if you wanna use only the central camera image
for line in lines:
    #print(line)
    # get the image path from the file
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = 'IMG/'+ filename
    #print('current_path', current_path)
    image = cv2.imread(current_path)
    images.append(image)
     # get the steering measurmement (labels) from the file
    measurment = line[3]
    measurments.append(float(measurment))
        
        
        
# convert features and labels to numpy array as that's the format keras requires  
print("Sample of labels", measurments[4823] )
#X_train = np.array(images)
#y_train = np.array(measurments) 
#print(X_train[0].shape, ' shape of train samples')
#print("size of training set", len(X_train))

from sklearn.utils import shuffle
# Shuffle training data, NOTE: No need for that part as it will be done in the model.fit()
#X_train, y_train = shuffle(X_train, y_train)

# Data Augmentation
augmented_images, augmented_measurments = [], []
for image,measurment in zip(images, measurments):
    augmented_images.append(image)
    augmented_measurments.append(measurment)
    augmented_images.append(cv2.flip(image, 1)) #flip image 180 horizontally
    augmented_measurments.append(measurment * -1.0) # reverse the steering angle
   



X_train = np.array(augmented_images)
y_train = np.array(augmented_measurments) 
# display sample image and it's corresponding label (steerng angle)
print(X_train[0].shape, ' shape of train samples')
#print(" a sample of label before flipping", y_train[4823])
#print(" a sample of label after  flipping", y_train[4824])
print("size of training set", len(X_train))

#print (" Subset y-labels are")
#print(y_train[0:1000])
    


# build a very simple model to test on it
import keras
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense  
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Lambda
from keras.layers import Cropping2D


model = Sequential()

# Preprocessing the data using Normalization and Mean-Center
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))

# Cropping un-useful details from the image to avoid distracting model training. Here I removed 50-pixels from top and 20 pixels from bottom.
model.add(Cropping2D(cropping=((70,25), (0,0))))

# Using LeNet5 model 
model.add(Convolution2D(6, 5, 5, activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6, 5, 5, activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.3, shuffle=True, epochs=5)

# save the model to reuse or download
model.save('model.h5')

# NOTE: by default Keras train for 10 epochs
# you will notice that before the 7th-epoch, the validation loss was decreasing then after it increased, so to prevent overfittng, we can nb_epoch = 7

print ('end')


## This section using Lenet5  images from the thre cameras with steering correction

In [ ]:
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.utils import shuffle

lines = []
i = 0
with open('driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        if(i > 0): #this part is added to insure the 1st line in the csvfile which include the headers is not included
             lines.append(line)
        i = i + 1

# create adjusted steering measurements for the side camera images
correction = 0.01 # this is a parameter to tune

images = []
measurments = []
# use this section if you wanna use images from the 3-cameras
for line in lines:
    #print(line)
    for x in range(3):
        # get the image path from the file
        source_path = line[x]
        filename = source_path.split('/')[-1]
        current_path = 'IMG/'+ filename
        #print('current_path', current_path)
        image = cv2.imread(current_path)
        images.append(image)
        # get the steering measurmement (labels) from the file
        measurment = line[3]
        if (x == 0):
            measurments.append(float(measurment))
        if (x == 1):
             measurments.append(float(measurment) + correction)
        if (x == 2):
             measurments.append(float(measurment) - correction)
            
            
  


        
        
        
# convert features and labels to numpy array as that's the format keras requires  
print("Sample of labels", measurments[4823] )
#X_train = np.array(images)
#y_train = np.array(measurments) 
#print(X_train[0].shape, ' shape of train samples')
#print("size of training set", len(X_train))

from sklearn.utils import shuffle
# Shuffle training data, NOTE: No need for that part as it will be done in the model.fit()
#X_train, y_train = shuffle(X_train, y_train)

# Data Augmentation
augmented_images, augmented_measurments = [], []
for image,measurment in zip(images, measurments):
    augmented_images.append(image)
    augmented_measurments.append(measurment)
    augmented_images.append(cv2.flip(image, 1)) #flip image 180 horizontally
    augmented_measurments.append(measurment * -1.0) # reverse the steering angle
   



X_train = np.array(augmented_images)
y_train = np.array(augmented_measurments) 
# display sample image and it's corresponding label (steerng angle)
print(X_train[0].shape, ' shape of train samples')
#print(" a sample of label before flipping", y_train[4823])
#print(" a sample of label after  flipping", y_train[4824])
print("size of training set", len(X_train))

#print (" Subset y-labels are")
#print(y_train[0:1000])
    


# build a very simple model to test on it
import keras
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense  
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Lambda
from keras.layers import Cropping2D


model = Sequential()

# Preprocessing the data using Normalization and Mean-Center
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))

# Cropping un-useful details from the image to avoid distracting model training. Here I removed 50-pixels from top and 20 pixels from bottom.
model.add(Cropping2D(cropping=((70,25), (0,0))))

# Using LeNet5 model 
model.add(Convolution2D(6, 5, 5, activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6, 5, 5, activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.3, shuffle=True, epochs=5)

# save the model to reuse or download
model.save('model.h5')

# NOTE: by default Keras train for 10 epochs
# you will notice that before the 7th-epoch, the validation loss was decreasing then after it increased, so to prevent overfittng, we can nb_epoch = 7

print ('end')


Sample of labels -0.23116130000000001
(160, 320, 3)  shape of train samples
size of training set 48216


Using TensorFlow backend.
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:103: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:105: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`


Train on 33751 samples, validate on 14465 samples
Epoch 1/5
   96/33751 [..............................] - ETA: 11:55 - loss: 3.3238